# Imports

In [1]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    SolverFactory,
    Suffix,
    value,
)
import idaes.core.solvers.get_solver
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.contrib.doe.utils import generate_snake_zigzag_pattern
from pyomo.contrib.parmest.experiment import Experiment
import pyomo.contrib.parmest.parmest as parmest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tee = True
x1_val = np.linspace(10, 50, 10)
x2_val = np.linspace(-10, 20, 10)
x3_val = np.linspace(60, 85, 10)

y = lambda x1, x2, x3: 0.5*x1 + 0.2*x2 + 0.3*x3

np.random.seed(123)
y_val = y(x1_val, x2_val, x3_val) + np.random.normal(0, 0.2, 10)

data = pd.DataFrame({'x1': x1_val, 'x2': x2_val, 'x3': x3_val, 'y': y_val})

data.head()

,x1,x2,x3,y
0,10.000000,-10.000000,60.000000,20.782874
1,14.444444,-6.666667,62.777778,24.921691
2,18.888889,-3.333333,65.555556,28.501040
3,23.333333,0.000000,68.333333,31.865408
4,27.777778,3.333333,71.111111,35.773169


# Experiment Class

In [3]:
class TestExample(Experiment):
    def __init__(self, data, theta_init=None):
        """
        Args:
            data: DataFrame
                A DataFrame containing the data to be used for the experiment.
                The DataFrame should have columns 'x1', 'x2', 'x3 and 'y'.
            theta_init: dict, optional
                A dictionary containing the initial values of the parameters.
                The keys should be 'a', 'b', and 'c'.
                If not provided, default values will be used.
        """
        self.data = data
        if theta_init is None:
            self.theta_init = {"a": 0.5, "b": 0.2, "c": 0.3}
        else:
            self.theta_init = theta_init
        self.model = None

    def get_labeled_model(self):
        if self.model is None:
            self.create_model()
            self.finalize_model()
            self.label_experiment()

        return self.model

    def create_model(self):
        m = self.model = ConcreteModel()
        m.x1 = Var(initialize=self.data.x1, bounds = (10, 50))
        m.x2 = Var(initialize=self.data.x2, bounds = (-10, 20))
        m.x3 = Var(initialize=self.data.x3, bounds = (60, 85))
        m.y = Var(initialize=self.data.y, bounds = (0, 100))

        m.a = Var(initialize=self.theta_init["a"], bounds = (0, 1))
        m.b = Var(initialize=self.theta_init["b"], bounds = (0, 1))
        m.c = Var(initialize=self.theta_init["c"], bounds = (0, 1))

        m.con = Constraint(expr=m.a * m.x1 + m.b * m.x2 + m.c * m.x3 == m.y)

    def finalize_model(self):
        m = self.model

        # fix the decision variables
        m.x1.fix()

        # print("~~~~~~~~~~~~~~~~~")
        # print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
        # print(m.x1.fixed)
        # print(m.x1.value)
        m.x2.fix()
        m.x3.fix()

        # fix the parameters
        m.a.fix()
        m.b.fix()
        m.c.fix()

    def label_experiment(self):
        m = self.model

        m.experiment_inputs = Suffix(direction=Suffix.LOCAL)
        # m.measurement_inputs.update()
        m.experiment_inputs[m.x1] = self.data.x1
        m.experiment_inputs[m.x2] = self.data.x2
        m.experiment_inputs[m.x3] = self.data.x3

        m.experiment_outputs = Suffix(direction=Suffix.LOCAL)
        m.experiment_outputs[m.y] = self.data.y

        m.unknown_parameters = Suffix(direction=Suffix.LOCAL)
        m.unknown_parameters[m.a] = self.theta_init["a"]
        m.unknown_parameters[m.b] = self.theta_init["b"]
        m.unknown_parameters[m.c] = self.theta_init["c"]

        m.measurement_error = Suffix(direction=Suffix.LOCAL)
        m.measurement_error[m.y] = 0.1

In [4]:
# exp_obj = TestExample(data.iloc[0])

In [5]:
# exp_obj.get_labeled_model()#.pprint()

In [6]:
# solver = SolverFactory("ipopt")
# solver.solve(exp_obj.model, tee=tee)

# Parmest

In [7]:
exp_list = []
for i in range(len(data)):
    exp_list.append(TestExample(data.iloc[i]))

pest = parmest.Estimator(exp_list, obj_function="SSE", tee = tee)

obj, theta = pest.theta_est()

print("Objective function value: ", obj)
print("Estimated parameters: ")
print(theta)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

## Plotting the experimental data and the predicted values

In [8]:
# y_pred = lambda x1, x2, x3 : theta["a"] * x1 + theta["b"] * x2 + theta["c"] * x3

In [9]:
# y_hat = y_pred(x1_val, x2_val, x3_val)

In [10]:
# plt.figure(figsize=(8, 8))
# plt.plot(y_val, y_hat, lw = 3, color = "r", label = "predicted_y vs observed_y")
# plt.plot(
#     (y_val.min(), y_val.max()), (y_val.min(), y_val.max()), "g--", lw=3, label="predicted_y = observed_y line"
# )
# plt.xlabel("observed_y", fontsize=16)
# plt.ylabel("predicted_y", fontsize=16)
# plt.title("Test 3P-3D", fontsize=16)

# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# plt.legend(fontsize=14)

# DOE

In [11]:
FIM = []
det_FIM = []
for i in range(len(data)):
    experiment = TestExample(data.iloc[i])
    doe_obj = DesignOfExperiments(experiment, scale_constant_value=True, tee=tee)
    FIM_i = doe_obj.compute_FIM()
    det_FIM.append(np.linalg.det(FIM_i))
    FIM.append(FIM_i)

FIM_total = np.array(FIM).sum(axis=0)
print("Fisher Information Matrix (FIM):")
print(FIM_total)

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

In [12]:
FIM_total

array([[106296.29629629,  27222.22222223, 227685.18518517],
       [ 27222.22222223,  11666.66666667,  43888.88888889],
       [227685.18518517,  43888.88888889, 531990.74074071]])

In [13]:
det_FIM

[np.float64(6.503348278683485e-22),
 np.float64(3.177313525013617e-23),
 np.float64(-5.372168610054538e-23),
 np.float64(0.0),
 np.float64(5.600506386469808e-22),
 np.float64(1.4604669315728737e-21),
 np.float64(-7.531287950607272e-21),
 np.float64(-6.728110627341295e-21),
 np.float64(-5.614320691583153e-20),
 np.float64(-1.4092775712178277e-19)]

In [14]:
eigval, eigvec =np.linalg.eigh(FIM_total)
print("Eigenvalues of FIM:", eigval)
print("Eigenvectors of FIM:")
print(eigvec)

Eigenvalues of FIM: [1.27201409e-11 1.49406375e+04 6.35013066e+05]
Eigenvectors of FIM:
[[-0.67168002 -0.62501582 -0.39774512]
 [ 0.7044449  -0.70503929 -0.08171278]
 [ 0.22935415  0.33507436 -0.91385001]]


In [15]:
np.linalg.det(FIM_total)

np.float64(-0.05721828891222297)

## Sensitivity analysis

In [18]:
design_ranges = {
    "x1": (15, 30, 3),
    "x2": (-5, 0, 3),
    "x3": (70, 80, 3),
}
experiment = TestExample(data.iloc[5])
doe_obj_ff = DesignOfExperiments(
    experiment, 
    scale_nominal_param_value=True, 
    tee=tee,
    prior_FIM=FIM_total
)

doe_obj_ff.compute_FIM_full_factorial(design_ranges= design_ranges)

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

{'x1': [np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0)],
 'x2': [np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(

# Standalone cell to compute sensitivity

In [ ]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    SolverFactory,
    Suffix,
    value,
)
import idaes.core.solvers.get_solver
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.contrib.doe.utils import generate_snake_zigzag_pattern
from pyomo.contrib.parmest.experiment import Experiment
import pyomo.contrib.parmest.parmest as parmest
import numpy as np
import pandas as pd

tee = True
x1_val = np.linspace(10, 50, 10)
x2_val = np.linspace(-10, 20, 10)
x3_val = np.linspace(60, 85, 10)

y = lambda x1, x2, x3: 0.5 * x1 + 0.2 * x2 + 0.3 * x3

np.random.seed(123)
y_val = y(x1_val, x2_val, x3_val) + np.random.normal(0, 0.2, 10)

data = pd.DataFrame({"x1": x1_val, "x2": x2_val, "x3": x3_val, "y": y_val})


class TestExample(Experiment):
    def __init__(self, data, theta_init=None):
        """
        Args:
            data: DataFrame
                A DataFrame containing the data to be used for the experiment.
                The DataFrame should have columns 'x1', 'x2', 'x3 and 'y'.
            theta_init: dict, optional
                A dictionary containing the initial values of the parameters.
                The keys should be 'a', 'b', and 'c'.
                If not provided, default values will be used.
        """
        self.data = data
        if theta_init is None:
            self.theta_init = {"a": 0.5, "b": 0.2, "c": 0.3}
        else:
            self.theta_init = theta_init
        self.model = None

    def get_labeled_model(self):
        if self.model is None:
            self.create_model()
            self.finalize_model()
            self.label_experiment()

        return self.model

    def create_model(self):
        m = self.model = ConcreteModel()
        m.x1 = Var(initialize=self.data.x1, bounds=(10, 50))
        m.x2 = Var(initialize=self.data.x2, bounds=(-10, 20))
        m.x3 = Var(initialize=self.data.x3, bounds=(60, 85))
        m.y = Var(initialize=self.data.y, bounds=(0, 100))

        m.a = Var(initialize=self.theta_init["a"], bounds=(0, 1))
        m.b = Var(initialize=self.theta_init["b"], bounds=(0, 1))
        m.c = Var(initialize=self.theta_init["c"], bounds=(0, 1))

        m.con = Constraint(expr=m.a * m.x1 + m.b * m.x2 + m.c * m.x3 == m.y)

    def finalize_model(self):
        m = self.model

        # fix the decision variables
        m.x1.fix()

        # print("~~~~~~~~~~~~~~~~~")
        # print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
        # print(m.x1.fixed)
        # print(m.x1.value)
        m.x2.fix()
        m.x3.fix()

        # fix the parameters
        m.a.fix()
        m.b.fix()
        m.c.fix()

    def label_experiment(self):
        m = self.model

        m.experiment_inputs = Suffix(direction=Suffix.LOCAL)
        # m.measurement_inputs.update()
        m.experiment_inputs[m.x1] = self.data.x1
        m.experiment_inputs[m.x2] = self.data.x2
        m.experiment_inputs[m.x3] = self.data.x3

        m.experiment_outputs = Suffix(direction=Suffix.LOCAL)
        m.experiment_outputs[m.y] = self.data.y

        m.unknown_parameters = Suffix(direction=Suffix.LOCAL)
        m.unknown_parameters[m.a] = self.theta_init["a"]
        m.unknown_parameters[m.b] = self.theta_init["b"]
        m.unknown_parameters[m.c] = self.theta_init["c"]

        m.measurement_error = Suffix(direction=Suffix.LOCAL)
        m.measurement_error[m.y] = 0.1


theta_init = {"a": 0.338325, "b": 0.370126, "c": 0.354424}
FIM_total = np.array(
    [
        [106296.29629629, 27222.22222223, 227685.18518517],
        [27222.22222223, 11666.66666667, 43888.88888889],
        [227685.18518517, 43888.88888889, 531990.74074071],
    ]
)
experiment = TestExample(data.iloc[5])
design_ranges = {
    "x1": (15, 30, 3),
    "x2": (-5, 0, 3),
    "x3": (70, 80, 3),
}

doe_obj_ff = DesignOfExperiments(
    experiment, scale_nominal_param_value=True, tee=tee, prior_FIM=FIM_total
)

doe_obj_ff.compute_FIM_full_factorial(design_ranges=design_ranges)

# print("FIM Full Factorial: \n")
# print(doe_obj_ff.fim_factorial_results)

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

{'x1': [np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(15.0),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(22.5),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0),
  np.float64(30.0)],
 'x2': [np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(-2.5),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-5.0),
  np.float64(-2.5),
  np.float64(

# New sensitivity initialization method

In [1]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    SolverFactory,
    Suffix,
    value,
)
import idaes.core.solvers.get_solver
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.contrib.doe.utils import generate_snake_zigzag_pattern
from pyomo.contrib.parmest.experiment import Experiment
import pyomo.contrib.parmest.parmest as parmest
import numpy as np
import pandas as pd

tee = True
x1_val = np.linspace(10, 50, 10)
x2_val = np.linspace(-10, 20, 10)
x3_val = np.linspace(60, 85, 10)

y = lambda x1, x2, x3: 0.5 * x1 + 0.2 * x2 + 0.3 * x3

np.random.seed(123)
y_val = y(x1_val, x2_val, x3_val) + np.random.normal(0, 0.2, 10)

data = pd.DataFrame({"x1": x1_val, "x2": x2_val, "x3": x3_val, "y": y_val})


class TestExample(Experiment):
    def __init__(self, data, theta_init=None):
        """
        Args:
            data: DataFrame
                A DataFrame containing the data to be used for the experiment.
                The DataFrame should have columns 'x1', 'x2', 'x3 and 'y'.
            theta_init: dict, optional
                A dictionary containing the initial values of the parameters.
                The keys should be 'a', 'b', and 'c'.
                If not provided, default values will be used.
        """
        self.data = data
        if theta_init is None:
            self.theta_init = {"a": 0.5, "b": 0.2, "c": 0.3}
        else:
            self.theta_init = theta_init
        self.model = None

    def get_labeled_model(self):
        if self.model is None:
            print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
            print("Creating model...")
            print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
            self.create_model()
            self.finalize_model()
            self.label_experiment()

        return self.model

    def create_model(self):
        m = self.model = ConcreteModel()
        m.x1 = Var(initialize=self.data.x1, bounds=(10, 50))
        m.x2 = Var(initialize=self.data.x2, bounds=(-10, 20))
        m.x3 = Var(initialize=self.data.x3, bounds=(60, 85))
        m.y = Var(initialize=self.data.y, bounds=(0, 100))

        m.a = Var(initialize=self.theta_init["a"], bounds=(0, 1))
        m.b = Var(initialize=self.theta_init["b"], bounds=(0, 1))
        m.c = Var(initialize=self.theta_init["c"], bounds=(0, 1))

        m.con = Constraint(expr=m.a * m.x1 + m.b * m.x2 + m.c * m.x3 == m.y)

    def finalize_model(self):
        m = self.model

        # fix the decision variables
        m.x1.fix()

        # print("~~~~~~~~~~~~~~~~~")
        # print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
        # print(m.x1.fixed)
        # print(m.x1.value)
        m.x2.fix()
        m.x3.fix()

        # fix the parameters
        m.a.fix()
        m.b.fix()
        m.c.fix()

    def label_experiment(self):
        m = self.model

        m.experiment_inputs = Suffix(direction=Suffix.LOCAL)
        # m.measurement_inputs.update()
        m.experiment_inputs[m.x1] = self.data.x1
        m.experiment_inputs[m.x2] = self.data.x2
        m.experiment_inputs[m.x3] = self.data.x3

        m.experiment_outputs = Suffix(direction=Suffix.LOCAL)
        m.experiment_outputs[m.y] = self.data.y

        m.unknown_parameters = Suffix(direction=Suffix.LOCAL)
        m.unknown_parameters[m.a] = self.theta_init["a"]
        m.unknown_parameters[m.b] = self.theta_init["b"]
        m.unknown_parameters[m.c] = self.theta_init["c"]

        m.measurement_error = Suffix(direction=Suffix.LOCAL)
        m.measurement_error[m.y] = 0.1


theta_init = {"a": 0.338325, "b": 0.370126, "c": 0.354424}
FIM_total = np.array(
    [
        [106296.29629629, 27222.22222223, 227685.18518517],
        [27222.22222223, 11666.66666667, 43888.88888889],
        [227685.18518517, 43888.88888889, 531990.74074071],
    ]
)
experiment = TestExample(data.iloc[5])
design_values = {
    "x1": [15, 30],
    "x2": [-5, 0],
    "x3": [70, 80],
}

doe_obj_ff = DesignOfExperiments(
    experiment, scale_nominal_param_value=True, tee=tee, prior_FIM=FIM_total
)

doe_obj_ff.compute_FIM_factorial(design_values=design_values)

# print("FIM Full Factorial: \n")
# print(doe_obj_ff.fim_factorial_results)























Creating model...






















design_keys: {'x3', 'x2', 'x1'}, 
map_keys: {'x3', 'x2', 'x1'}
des_ranges: [[15, 30], [-5, 0], [70, 80]]
factorial_points: [(15, -5, 70), (15, -5, 80), (15, 0, 80), (15, 0, 70), (30, 0, 80), (30, 0, 70), (30, -5, 70), (30, -5, 80)]
Total points: 8
design_point: (15, -5, 70)
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/

{'x1': [15, 15, 15, 15, 30, 30, 30, 30],
 'x2': [-5, -5, 0, 0, 0, 0, -5, -5],
 'x3': [70, 80, 80, 70, 80, 70, 70, 80],
 'log10 D-opt': [np.float64(10.910005833718511),
  np.float64(9.728644938785475),
  np.float64(10.315598657320967),
  np.float64(9.666576470724989),
  np.float64(12.297117344880036),
  np.float64(12.41891900314521),
  np.float64(12.528111705635148),
  np.float64(12.421620681726909)],
 'log10 A-opt': [np.float64(5.8161989981156985),
  np.float64(5.81709327471503),
  np.float64(5.817086657209006),
  np.float64(5.816192366969107),
  np.float64(5.818201936598775),
  np.float64(5.817309942329936),
  np.float64(5.817316556434583),
  np.float64(5.818208537132804)],
 'log10 E-opt': [np.float64(0.9268451554985807),
  np.float64(-0.25742228039568793),
  np.float64(0.3309947985480839),
  np.float64(-0.3155137189014248),
  np.float64(2.3141556614087007),
  np.float64(2.435857993436513),
  np.float64(2.545892510549663),
  np.float64(2.439091115875269)],
 'log10 ME-opt': [np.float64

In [1]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    SolverFactory,
    Suffix,
    value,
)
import idaes.core.solvers.get_solver
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.contrib.doe.utils import generate_snake_zigzag_pattern
from pyomo.contrib.parmest.experiment import Experiment
import pyomo.contrib.parmest.parmest as parmest
import numpy as np
import pandas as pd

tee = True
x1_val = np.linspace(10, 50, 10)
x2_val = np.linspace(-10, 20, 10)
x3_val = np.linspace(60, 85, 10)

y = lambda x1, x2, x3: 0.5 * x1 + 0.2 * x2 + 0.3 * x3

np.random.seed(123)
y_val = y(x1_val, x2_val, x3_val) + np.random.normal(0, 0.2, 10)

data = pd.DataFrame({"x1": x1_val, "x2": x2_val, "x3": x3_val, "y": y_val})


class TestExample(Experiment):
    def __init__(self, data, theta_init=None):
        """
        Args:
            data: DataFrame
                A DataFrame containing the data to be used for the experiment.
                The DataFrame should have columns 'x1', 'x2', 'x3 and 'y'.
            theta_init: dict, optional
                A dictionary containing the initial values of the parameters.
                The keys should be 'a', 'b', and 'c'.
                If not provided, default values will be used.
        """
        self.data = data
        if theta_init is None:
            self.theta_init = {"a": 0.5, "b": 0.2, "c": 0.3}
        else:
            self.theta_init = theta_init
        self.model = None

    def get_labeled_model(self):
        if self.model is None:
            print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
            print("Creating model...")
            print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
            self.create_model()
            self.finalize_model()
            self.label_experiment()

        return self.model

    def create_model(self):
        m = self.model = ConcreteModel()
        m.x1 = Var(initialize=self.data.x1, bounds=(10, 50))
        m.x2 = Var(initialize=self.data.x2, bounds=(-10, 20))
        m.x3 = Var(initialize=self.data.x3, bounds=(60, 85))
        m.y = Var(initialize=self.data.y, bounds=(0, 100))

        m.a = Var(initialize=self.theta_init["a"], bounds=(0, 1))
        m.b = Var(initialize=self.theta_init["b"], bounds=(0, 1))
        m.c = Var(initialize=self.theta_init["c"], bounds=(0, 1))

        m.con = Constraint(expr=m.a * m.x1 + m.b * m.x2 + m.c * m.x3 == m.y)

    def finalize_model(self):
        m = self.model

        # fix the decision variables
        m.x1.fix()

        # print("~~~~~~~~~~~~~~~~~")
        # print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
        # print(m.x1.fixed)
        # print(m.x1.value)
        m.x2.fix()
        m.x3.fix()

        # fix the parameters
        m.a.fix()
        m.b.fix()
        m.c.fix()

    def label_experiment(self):
        m = self.model

        m.experiment_inputs = Suffix(direction=Suffix.LOCAL)
        # m.measurement_inputs.update()
        m.experiment_inputs[m.x1] = self.data.x1
        m.experiment_inputs[m.x2] = self.data.x2
        m.experiment_inputs[m.x3] = self.data.x3

        m.experiment_outputs = Suffix(direction=Suffix.LOCAL)
        m.experiment_outputs[m.y] = self.data.y

        m.unknown_parameters = Suffix(direction=Suffix.LOCAL)
        m.unknown_parameters[m.a] = self.theta_init["a"]
        m.unknown_parameters[m.b] = self.theta_init["b"]
        m.unknown_parameters[m.c] = self.theta_init["c"]

        m.measurement_error = Suffix(direction=Suffix.LOCAL)
        m.measurement_error[m.y] = 0.1


theta_init = {"a": 0.338325, "b": 0.370126, "c": 0.354424}
FIM_total = np.array(
    [
        [106296.29629629, 27222.22222223, 227685.18518517],
        [27222.22222223, 11666.66666667, 43888.88888889],
        [227685.18518517, 43888.88888889, 531990.74074071],
    ]
)
experiment = TestExample(data.iloc[5])
design_values = {
    "x1": [15, 30],
    "x2": [-5, 0],
    "x3": [70, 80],
}

doe_obj_ff = DesignOfExperiments(
    experiment, scale_nominal_param_value=True, tee=tee, prior_FIM=FIM_total
)

doe_obj_ff.compute_FIM_factorial(design_values=design_values, file_name= "test_file")

# print("FIM Full Factorial: \n")
# print(doe_obj_ff.fim_factorial_results)























Creating model...






















design_keys: {'x3', 'x1', 'x2'}, 
map_keys: {'x3', 'x1', 'x2'}
des_ranges: [[15, 30], [-5, 0], [70, 80]]
factorial_points: [(15, -5, 70), (15, -5, 80), (15, 0, 80), (15, 0, 70), (30, 0, 80), (30, 0, 70), (30, -5, 70), (30, -5, 80)]
Total points: 8
design_point: (15, -5, 70)
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/

{'x1': [15, 15, 15, 15, 30, 30, 30, 30],
 'x2': [-5, -5, 0, 0, 0, 0, -5, -5],
 'x3': [70, 80, 80, 70, 80, 70, 70, 80],
 'log10 D-opt': [np.float64(10.910005833718511),
  np.float64(9.728644938785475),
  np.float64(10.315598657320967),
  np.float64(9.666576470724989),
  np.float64(12.297117344880036),
  np.float64(12.41891900314521),
  np.float64(12.528111705635148),
  np.float64(12.421620681726909)],
 'log10 A-opt': [np.float64(5.8161989981156985),
  np.float64(5.81709327471503),
  np.float64(5.817086657209006),
  np.float64(5.816192366969107),
  np.float64(5.818201936598775),
  np.float64(5.817309942329936),
  np.float64(5.817316556434583),
  np.float64(5.818208537132804)],
 'log10 E-opt': [np.float64(0.9268451554985807),
  np.float64(-0.25742228039568793),
  np.float64(0.3309947985480839),
  np.float64(-0.3155137189014248),
  np.float64(2.3141556614087007),
  np.float64(2.435857993436513),
  np.float64(2.545892510549663),
  np.float64(2.439091115875269)],
 'log10 ME-opt': [np.float64